In [ ]:
!pip show langchain

In [ ]:
from langchain_ollama.llms import OllamaLLM

model = OllamaLLM(model="deepseek-r1:1.5b")

In [ ]:
from typing import List, Dict
from langgraph.graph import StateGraph, START, END

In [ ]:
# Step 1: Define State
class State(Dict):
    messages: List[Dict[str, str]] 


# Step 2: Initialize StateGraph
graph_builder = StateGraph(State)

llm = OllamaLLM(model="deepseek-r1:1.5b")

In [ ]:
# Define chatbot function
def chatbot(state: State):
    response = llm.invoke(state["messages"])
    state["messages"].append({"role": "assistant", "content": response})  # Treat response as a string
    return {"messages": state["messages"]}

In [ ]:
# Add nodes and edges
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

# Compile the graph
graph = graph_builder.compile()

In [ ]:
# Stream updates
def stream_graph_updates(user_input: str):    
    state = {"messages": [{"role": "user", "content": user_input}]}
    for event in graph.stream(state):
        for value in event.values():
            print("Assistant:", value["messages"][-1]["content"])

In [ ]:
# Run chatbot in a loop
if __name__ == "__main__":
    while True:
        try:
            user_input = input("You: ")
            if user_input.lower() in ["quit", "exit", "q"]:
                print("Happy to help!!")
                break

            stream_graph_updates(user_input)
        except Exception as e:
            print(f"An error occurred: {e}")
            break